In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
#from torchvision.datasets import MNIST

In [2]:
pip install -U matplotlib

Requirement already up-to-date: matplotlib in c:\apps\anaconda3\envs\gpu-cuda10\lib\site-packages (3.3.2)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
print(torch.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
#import os
#os.chdir("/content/drive/My Drive") 
import ra
#import complexLayers


In [4]:
dataset = ra.read('atomsroa.ra')
train_data = torch.tensor(dataset)

In [5]:
train_data = train_data.imag
train_data =  torch.transpose(train_data, 0, 1).unsqueeze(1)

In [6]:
labelset = ra.read('paramsroa.ra')
train_label1 = torch.tensor(labelset)

In [7]:
dataset1 = ra.read('atoms.ra')
traindata1 = torch.tensor(dataset1)
traindata1 = traindata1.imag
traindata1 =  torch.transpose(traindata1, 0, 1).unsqueeze(1)
label1 = ra.read('params.ra')
label1 = torch.tensor(label1)

In [8]:
print(traindata1)
print(train_data)

tensor([[[ 0.0038,  0.0015, -0.0007,  ..., -0.0274, -0.0272, -0.0269]],

        [[ 0.0045,  0.0023,  0.0001,  ..., -0.0279, -0.0276, -0.0273]],

        [[ 0.0052,  0.0031,  0.0010,  ..., -0.0283, -0.0280, -0.0278]],

        ...,

        [[ 0.0215,  0.0229,  0.0242,  ..., -0.0250, -0.0246, -0.0243]],

        [[ 0.0216,  0.0230,  0.0243,  ..., -0.0250, -0.0246, -0.0243]],

        [[ 0.0216,  0.0230,  0.0243,  ..., -0.0250, -0.0247, -0.0243]]],
       dtype=torch.float64)
tensor([[[-0.0100, -0.0114, -0.0094,  ...,  0.0324,  0.0322,  0.0320]],

        [[-0.0105, -0.0123, -0.0106,  ...,  0.0329,  0.0327,  0.0325]],

        [[-0.0110, -0.0132, -0.0117,  ...,  0.0333,  0.0331,  0.0330]],

        ...,

        [[-0.0141, -0.0290, -0.0443,  ...,  0.0006,  0.0012,  0.0019]],

        [[-0.0141, -0.0290, -0.0443,  ...,  0.0006,  0.0012,  0.0019]],

        [[-0.0141, -0.0289, -0.0443,  ...,  0.0006,  0.0012,  0.0019]]],
       dtype=torch.float64)


In [11]:
print(traindata1.shape)
train_data.shape

torch.Size([108056, 1, 1000])


torch.Size([108056, 1, 1000])

In [9]:
alldata = torch.cat((train_data,traindata1),0)

In [10]:
labels1 = torch.cat((train_label1[:-1,:],label1[:-1,:]),1)

In [ ]:
print(labels1)
print(labels1.shape)
print(train_label1.shape)

In [11]:
labels1 = torch.transpose(labels1, 0, 1)
print(labels1.shape)

torch.Size([216112, 2])


In [12]:
import torch.utils.data as data
train_dataset = data.TensorDataset(alldata,labels1)

In [ ]:
from sklearn.decomposition import PCA
X_reduced = PCA(n_components=10).fit_transform(train_data)

In [ ]:
pca = PCA(n_components=10)
pca.fit(train_data)

print(pca.explained_variance_ratio_.shape)
print(pca.components_.shape)
print(pca.singular_values_.shape)
print(pca.mean_.shape)

In [ ]:
train_data = torch.tensor(X_reduced)

In [13]:
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(train_dataset, [140000, 60000 ,16112])

We found that the network model with smallest
average error for validation data consists of 3 convolutional layers (kernel size = 3, stride
size = 2), each followed by a rectified linear unit (ReLU) activation function. The number
of the feature maps per convolutional layer is increasing, from 32 in the first to 128 in
the last. After convolution an average pooling layer follows with the same size as the
stride size. The last layer is fully connected, with 2 outputs. 

In [14]:
class AlexNet(nn.Module):

    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Conv1d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Conv1d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            #nn.Dropout(),
            nn.Linear(36, 18),
            nn.ReLU(inplace=True),
            #nn.Dropout(),
            nn.Linear(18, 9),
            nn.ReLU(inplace=True),
            nn.Linear(9, 2),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [16]:
N = 1000
val_loss = list()
#val_accu = list()
train_loss = list()
#train_accu = list()

In [15]:
#assert torch.cuda.is_available()
device1 = "cuda:0" if torch.cuda.is_available() else "cpu"
#cuda_device = torch.device("cuda")
seed = 7
torch.manual_seed(seed)
trainloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
validloader = DataLoader(valid_dataset, batch_size=128, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

def loss_batch(model, loss_func, xb, yb, opt=None):
    outputs = model(xb.float())
    yb = yb.float()
    outputs = torch.squeeze(outputs, 1)
    loss = loss_func(outputs, yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

In [ ]:
!pip install pycuda
import torch
import pycuda.driver as cuda
cuda.init()
## Get Id of default device
torch.cuda.current_device()
# 0
cuda.Device(0).name() # '0' is the id of your GPU

In [ ]:

# build the model
model = AlexNet().to(device1)

# define the loss function and the optimiser
loss_function = nn.MSELoss()
optimiser = optim.Adam(model.parameters())

# the epoch loop
for epoch in range(N):
    model.train()
    running_loss = 0.0
    loss=0
    for data in trainloader:
        # get the inputs
        inputs, labels = data
        #inputs = inputs.unsqueeze(1).unsqueeze(-1)
        #inputs = inputs.unsqueeze(1)
        #inputs = torch.unsqueeze(inputs1, 2)

        inputs, labels = inputs.to(device1), labels.to(device1)
        
        
        loss, nums = loss_batch(model, loss_function, inputs.float(), labels.float(), optimiser)
        running_loss += loss
    train_loss.append(loss)

    model.eval()
    with torch.no_grad():
        losses, nums = zip(
                *[loss_batch(model.to(device1), loss_function, xb.to(device1), yb.to(device1)) for xb, yb in validloader]
            )
        val_loss1 = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        val_loss.append(val_loss1)


        #print(epoch, val_loss)

        
        # keep track of the loss this epoch
        #running_loss += loss.item()
    print("Epoch %d, loss %4.2f" % (epoch, running_loss))
print('**** Finished Training ****')

In [ ]:
import matplotlib.pyplot as plt

x = range(0, N)
y1 = train_loss
#y2 = train_accu
y3 = val_loss
#y4 = val_accu
plt.subplot(2, 1, 1)
plt.plot(x, y1, 'o-')
plt.title('train_loss and train_accu')
plt.ylabel('train_loss')
#plt.subplot(2, 1, 2)
#plt.plot(x, y2, '.-')
#plt.xlabel('epoches')
#plt.ylabel('train_accu')
plt.subplot(2, 1, 2)
plt.plot(x, y3, 'o-')
plt.ylabel('val_loss')
#plt.subplot(2, 2, 2)
#plt.plot(x, y4, '.-')
#plt.title('val_loss and val_accu')
#lt.ylabel('val_accu')
plt.show()
#plt.savefig("accuracy_loss.jpg")


In [ ]:
print(y1)

In [ ]:
print(y3)

In [ ]:
model.eval()

In [19]:
model.eval()
test_loss = list()
for data in testloader:
    losses, nums = zip(
                *[loss_batch(model.to(device1), loss_function, xb.to(device1), yb.to(device1)) for xb, yb in testloader])
    test_loss1 = np.sum(np.multiply(losses, nums)) / np.sum(nums)
    test_loss.append(test_loss1)

In [20]:
print(test_loss)

[17.467445583779146, 17.467445481498288, 17.467445536426897, 17.467445456875115, 17.467445589461416, 17.467445526956446, 17.467445587567326, 17.467445526956446, 17.467445727729988, 17.467445568626427, 17.467445682271826, 17.467445633025488, 17.467445572414608, 17.467445566732337, 17.467445417099228, 17.467445558208933, 17.467445456875115, 17.467445646284116, 17.467445522221222, 17.467445631131397, 17.467445617872766, 17.467445451192848, 17.467445634919578, 17.467445557261886, 17.467445420887405, 17.467445509909638, 17.467445646284116, 17.46744556957347, 17.467445528850536, 17.467445589461416, 17.467445559155976, 17.467445553473706, 17.467445436040126, 17.467445502333277, 17.467445574308698, 17.467445604614138, 17.467445472027837, 17.467445481498288, 17.467445459716252, 17.46744556767938, 17.467445436040126, 17.467445489074645, 17.467445509909638, 17.467445487180555, 17.467445608402315, 17.467445417099228, 17.467445398158326, 17.467445591355506, 17.467445612190495, 17.467445487180555, 1

In [2]:
import matplotlib.pyplot as plt

In [ ]:
x = range(0, 126)
y3 = test_loss
plt.plot(x, y3, 'o-')
plt.title('test_loss')
plt.ylabel('test_loss')

In [ ]:
for data in testloader:
    inputs, lablels = data
    predictions = model(inputs.float())
    loss = loss_function(predictions,labels)
    loss = loss/128
    print("loss",loss)
    print("predicted parameter:", predictions)
    print("setting parameter:", labels)

In [ ]:
testdataset = ra.read('atomsroa.ra')
testdata = torch.tensor(testdataset)
testdata = testdata.imag
label1 = ra.read('paramsroa.ra')
label1 = torch.tensor(label1)

In [ ]:
for i in range(8056):
    inputs = testdata[:,i]
    lablels = label1[0,i]
    predictions = model(inputs.float())

    print("predicted parameter:", predictions)
    print("setting parameter:", lablels)

## Saving models

Being able to train a model is fine, but in practice once we've trained the model we probably want to save the result so we can reuse it at a later time. PyTorch makes saving the model easy using the torch.save(state, filepath) function. This will save the weights of the model so they can be loaded into a new instance at a later point.

Run the following code to save the weights for use in the next part of the lab.

In [17]:
#save the trained model weights
torch.save(model.state_dict(), "./Alexnet.weights")

If you are running on Colab, run the following to download the weights to the local machine:

In [ ]:
from google.colab import files
files.download('BaselineModel.weights')

In [17]:
model = AlexNet().to(device1)

# define the loss function and the optimiser
loss_function = nn.MSELoss()
optimiser = optim.Adam(model.parameters())
model.load_state_dict(torch.load('Alexnet.weights'))

<All keys matched successfully>